# Criando o Modelo de Machine Learning

In [139]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

In [140]:
dataset = pd.read_json("clean_dataset.json")

SEED = 0
np.random.seed(SEED)

### Reajustando o dataset para o modelo de ML
---

In [141]:
# Categoria Desnecessária
dataset = dataset.drop(["ID", "custo_ate_entao"], axis = 1)

# Mudando para 0s e 1s
dataset = dataset.replace(['Sim', 'Nao'], [1, 0])
dataset = dataset.replace(['Sem Servico Internet', 'Sem Servico Telefone'], [0, 0])
dataset = dataset.replace(['Masculino', 'Feminino'], [1, 0])

# Aplicando Encoding nos outros dados
dataset = pd.get_dummies(data = dataset, columns = ['servico_internet', 'tipo_contrato', 'forma_pagamento'])
dataset = dataset.drop(["servico_internet_0"], axis=1)

### Analisando o Churn
---

In [142]:
dataset.query("deixou_empresa == 1").shape, dataset.query("deixou_empresa == 0").shape

((1869, 26), (5174, 26))

##### Como já verificado posteriormente, temos uma discrepência a respeito da taxa de churn. Mais pessoas escolheram permanecer assinando o serviço do que desistir do mesmo, o que pode gerar problemas em nosso futuro modelo. Vamos aplicar o SMOTE para resolver tal problema.

In [143]:
x = dataset.drop(["deixou_empresa"], axis = 1)
y = dataset.deixou_empresa

In [144]:
smote = SMOTE()
x_new_sample, y_new_sample = smote.fit_resample(x, y)
resampled_dataset = pd.concat([x_new_sample, y_new_sample], axis = 1)

### Escolhendo Modelos
---

In [145]:
cross_validation = KFold(n_splits = 10)

In [146]:
def valida_modelo(modelo):
    result = cross_validate(modelo, x_new_sample, y_new_sample, cv=cross_validation, 
                              scoring=['accuracy', 'precision', 'recall', 'f1'])
    print(f'\nModelo: {modelo}')
    print(f"A acurácia do modelo é {result['test_accuracy'].mean() * 100 :.2f} %")
    print(f"A precisão do modelo é {result['test_precision'].mean() * 100 :.2f} %")
    print(f"O recall do modelo é {result['test_recall'].mean() * 100 :.2f} %")
    print(f"O f1 do modelo é {result['test_f1'].mean() * 100 :.2f} %")

##### Iniciando pela Baseline

In [147]:
dummy = DummyClassifier()
valida_modelo(dummy)



Modelo: DummyClassifier()
A acurácia do modelo é 20.01 %
A precisão do modelo é 20.01 %
O recall do modelo é 70.00 %
O f1 do modelo é 30.99 %


##### Testando Quatro Modelos Diferentes de ML

In [148]:
knn_model = KNeighborsClassifier()
tree_model = DecisionTreeClassifier()
bagging_model = RandomForestClassifier()
boosting_model = AdaBoostClassifier()

valida_modelo(knn_model)
valida_modelo(tree_model)
valida_modelo(bagging_model)
valida_modelo(boosting_model)


Modelo: KNeighborsClassifier()
A acurácia do modelo é 79.73 %
A precisão do modelo é 66.57 %
O recall do modelo é 89.61 %
O f1 do modelo é 74.76 %

Modelo: DecisionTreeClassifier()
A acurácia do modelo é 80.59 %
A precisão do modelo é 68.63 %
O recall do modelo é 72.64 %
O f1 do modelo é 70.33 %

Modelo: RandomForestClassifier()
A acurácia do modelo é 84.71 %
A precisão do modelo é 73.60 %
O recall do modelo é 76.78 %
O f1 do modelo é 75.07 %

Modelo: AdaBoostClassifier()
A acurácia do modelo é 82.18 %
A precisão do modelo é 70.73 %
O recall do modelo é 79.20 %
O f1 do modelo é 74.24 %


##### Os algoritmos de árvore de decisão que fazem uso do ensambled learning apresentam uma performance acima do algoritmo padrão, logo, o DecisionTreeClassifier será desconsiderado.

##### Em relação a precisão e assertividade dos modelos, o KNN apresenta o melhor recall, o AdaBoost tem um resultado mais mediano e o RandomForest apresenta a melhor precisão. Como o f1 de todos é similar, vamos tentar otimizar os modelos e observar os resultados

### Otimizando os Hiperparâmetros
---

##### Usando o GridSearchCV para Otimização

In [165]:
# KNN
parameters_dictionary = {
    "n_neighbors": [5, 10, 20],
    "weights": ['uniform', 'distance'],
    "algorithm": ['ball_tree', 'kd_tree', 'brute', 'auto'],
    "p": [1, 2]
}

search = GridSearchCV(KNeighborsClassifier(), parameters_dictionary, cv=cross_validation, n_jobs=-1)
search.fit(x_new_sample, y_new_sample)
print(search.best_estimator_)

KNeighborsClassifier(algorithm='ball_tree', p=1, weights='distance')


In [157]:
# Random Forest
parameters_dictionary = {
    "n_estimators": [50, 100, 150, 200],
    "criterion": ["gini", "entropy", "log_loss"], 
    "max_features": ["sqrt", "log2"],
    "max_depth": [5, 10], 
    "min_samples_split": [5, 10],
    "bootstrap": [True, False]
}

search = GridSearchCV(RandomForestClassifier(), parameters_dictionary, cv=cross_validation, n_jobs=-1)
search.fit(x_new_sample, y_new_sample)
print(search.best_estimator_)

RandomForestClassifier(bootstrap=False, max_depth=10, min_samples_split=5,
                       n_estimators=150)


In [166]:
# Ada Boost
parameters_dictionary = {
    "n_estimators": [50, 100, 150, 200],
    "algorithm": ["SAMME", "SAMME.R"],
    
}

search = GridSearchCV(AdaBoostClassifier(), parameters_dictionary, cv=cross_validation, n_jobs=-1)
search.fit(x_new_sample, y_new_sample)
print(search.best_estimator_)

AdaBoostClassifier(n_estimators=200)


In [159]:
knn_model = KNeighborsClassifier(algorithm='ball_tree', p=1, weights='distance')
bagging_model = RandomForestClassifier(bootstrap=False, max_depth=10, min_samples_split=5, n_estimators=150)
boosting_model = AdaBoostClassifier(n_estimators=200)

valida_modelo(knn_model)
valida_modelo(bagging_model)
valida_modelo(boosting_model)


Modelo: KNeighborsClassifier(algorithm='ball_tree', p=1, weights='distance')
A acurácia do modelo é 82.13 %
A precisão do modelo é 68.32 %
O recall do modelo é 91.00 %
O f1 do modelo é 76.73 %

Modelo: RandomForestClassifier(bootstrap=False, max_depth=10, min_samples_split=5,
                       n_estimators=150)
A acurácia do modelo é 83.77 %
A precisão do modelo é 71.57 %
O recall do modelo é 81.77 %
O f1 do modelo é 75.91 %

Modelo: AdaBoostClassifier(n_estimators=200)
A acurácia do modelo é 83.64 %
A precisão do modelo é 72.28 %
O recall do modelo é 77.95 %
O f1 do modelo é 74.77 %


### Conclusão
---

##### Analisando a otimização básica de três modelos, podemos chegar as seguintes conclusões:

##### 1 - O modelo KNN apresenta o melhor desempenho geral, com F1 de 76% e recall acima de 90%

##### 2 - O modelo Ada Boosting apresenta a melhor precisão dos modelos, com 72%

##### Tudo então, irá depender dos recursos disponíveis a empresa, e quão disposta a mesma está a investir nesses resultados. Se a empresa não tiver recursos suficientes para buscar analisar a situação de cada um dos clientes indicados pelo modelo, um bom recall seria desejável, mesmo a custo da precisão. No entanto, caso o objetivo seja somente atingir o máximo de clientes possível, o Ada Boosting se mostra mais promissor.